In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from globalVariables import *
import simpleCNNwithInception as inceptionCNN
import simpleCNN as simpleCNN
import simpleCNNwithInceptionRegression as regressor
from keras.callbacks import CSVLogger
from scipy.ndimage import rotate

Objectives
----------
1. To see if transfer learning the DMO simulations before training on the bayrons improves it

Conclusions
-----------



#### 1. Get the fiducial for total and stellar

In [ ]:
fiducial_train_ds, fiducial_test_ds  = get_tf_DataSet(augment_data=True, 
                                            simulationNames=['CDM','SIDM1'], channels=['total','stellar'])
fiducial_model_2_channel = simpleCNN.getModel_ds(fiducial_train_ds.num_classes)
epochs=50
history = fiducial_model_2_channel.fit(
  x=fiducial_train_ds,
  validation_data=fiducial_test_ds,
  epochs=epochs
)

#### 2. Get the individual models

In [ ]:
fiducial_train_ds, fiducial_test_ds  = get_tf_DataSet(augment_data=True, 
                                simulationNames=['CDM','SIDM1'], channels=['total'])
total_model = simpleCNN.getModel_ds(fiducial_train_ds.num_classes)
epochs=50
history = total_model.fit(
  x=fiducial_train_ds,
  validation_data=fiducial_test_ds,
  epochs=epochs
)

In [ ]:
fiducial_train_ds, fiducial_test_ds  = get_tf_DataSet(augment_data=True, 
                                simulationNames=['CDM','SIDM1'], channels=['stellar'])
stellar_model = simpleCNN.getModel_ds(fiducial_train_ds.num_classes)
epochs=50
history = stellar_model.fit(
  x=fiducial_train_ds,
  validation_data=fiducial_test_ds,
  epochs=epochs
)

In [14]:
cat_1 = stellar_model.output
cat_2 = total_model.output
concat = tf.keras.layers.Concatenate()([cat_1, cat_2])

In [17]:
concat

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'concatenate_2')>

#### Results

In [ ]:
plt.figure(figsize=(7,7))

models =[fiducial_model, transfer_model, fiducial_model_2_channel, transfer_model_2_channel]
names = ['fid','transfer','fid_2','transfer_2']
c = ['r','b','g','c','y','k']
for i, iModel in enumerate(models):
    plt.plot(iModel.history.epoch, iModel.history.history['val_accuracy'], label=names[i], c=c[i])
    #plt.plot(iModel.history.epoch, [np.max(iModel.history.history['val_accuracy'])]*len(iModel.history.epoch), c=c[i])

plt.legend()